<a href="https://colab.research.google.com/github/Vurimindi2021/Capstone2/blob/main/image_test_train2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import math
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras import layers, Model, optimizers, callbacks
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tensorflow.keras.metrics import BinaryAccuracy, Recall, Precision, SensitivityAtSpecificity
from tensorflow.keras.metrics import FalseNegatives, FalsePositives
#from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19 
#from tensorflow.keras.preprocessing.image import image, load_img, img_to_array
from numpy import expand_dims
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.utils import plot_model

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
#Paths 
train_path = '/content/drive/MyDrive/output/train'
valid_path = '/content/drive/MyDrive/output/val'
test_path  = '/content/drive/MyDrive/output/test'
chkpt_path = '/content/drive/MyDrive/check_point2'
log_path = os.path.join("/content/drive/MyDrive/logs/fit/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
#Global Parameters
types       = ['Atelectasis','NoFinding']
batch_size  = 32
epochs      = 50

In [ ]:
import math
train_size  = 15177
val_size    = 3702
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))
steps_per_epoch = compute_steps_per_epoch(train_size)
val_steps = compute_steps_per_epoch(val_size)

In [ ]:
#ImageDataGenerator
train_images = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 5, 
                                  shear_range = 0.02,
                                  zoom_range = 0.02, 
                                  samplewise_center=True, 
                                  samplewise_std_normalization= True
                                  ).flow_from_directory(train_path, 
                                                        target_size=(256,256),
                                                        classes=types, 
                                                        batch_size=batch_size,
                                                        class_mode='binary')                               
valid_images = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 5, 
                                  shear_range = 0.02,
                                  zoom_range = 0.02, 
                                  samplewise_center=True, 
                                  samplewise_std_normalization= True
                                  ).flow_from_directory(valid_path, 
                                                        target_size=(256,256),
                                                        classes=types, 
                                                        batch_size=batch_size,
                                                        class_mode='binary')  
test_images = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path, 
                                                        target_size=(256,256),
                                                        classes=types, 
                                                        batch_size=3717,
                                                        class_mode='binary')

In [ ]:
# load model
model = VGG19(weights='imagenet', include_top=False,  input_shape=(256,256,3))

In [ ]:
# adding layers 
for layer in model.layers: 
    layer.trainable = False
x = tf.keras.layers.Flatten()(model.output)
x = tf.keras.layers.Dense(100, activation='relu')(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_vvg_random = tf.keras.models.Model(inputs=model.input, outputs=predictions)

In [ ]:
# compile
model_vvg_random.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001), 
                         loss='binary_crossentropy',               
                         metrics=[
                                  BinaryAccuracy(), 
                                  Recall(), 
                                  Precision()
                                  ])
# summarize the model
model_vvg_random.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='/content/drive/MyDrive/figs/vgg19_model1.png')

In [ ]:
checkpoint = callbacks.ModelCheckpoint(filepath=chkpt_path,
                                    save_weights_only=True,
                                    monitor='val_binary_accuracy',
                                    mode='max',
                                    save_best_only=True)
earlystop = callbacks.EarlyStopping(monitor='val_binary_accuracy', 
                                    min_delta=0.001, 
                                    patience=5, 
                                    verbose=0,
                                    mode='auto', 
                                    baseline=None, 
                                    restore_best_weights=True)
tensorboard = callbacks.TensorBoard(log_dir=log_path, 
                                    histogram_freq=1,
                                    write_graph=True)
# train the model
model_vvg_random.fit(train_images,
                      shuffle=True,
                      epochs=epochs,
                      steps_per_epoch=steps_per_epoch,
                      validation_steps=val_steps,
                      validation_data=valid_images,
                      callbacks=[checkpoint, earlystop, tensorboard,cm_callback])

In [ ]:
# Calculate the class probabilities
probabilities = model_vvg_random.predict(test_images)
classes = probabilities> 0.5
len(classes)

In [ ]:
print(classes)

In [ ]:
%load_ext tensorboard
#%reload_ext tensorboard
import tensorflow as tf
import datetime
%tensorboard --logdir /content/drive/MyDrive/logs